In [ ]:
%matplotlib inline
import tidy3d.web as web
import tidy3d as td
import trimesh as tri 
from tidy3d.plugins.resonance import ResonanceFinder
from pathlib import Path
from tidy3d.components.data.data_array import SpatialDataArray
import h5py
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import numpy as np
from pathlib import Path
from stl import mesh
import matplotlib.pyplot as plt

In [ ]:
import sys
import os

# Assuming /AutomationModule is in tahe root directory of your project
sys.path.append(os.path.abspath(fr'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]


In [ ]:
lambdas = np.array([8,3])
lambdas

In [ ]:
rng = np.random.default_rng(12345) #randon number seeds

filename = rf"H:\phd stuff\tidy3d\structures\LSU H5\20250627 eps 10p89\ak4_1000_ends_rad_0.695_eps_ff_0.2325.h5"
project_name = "20250704 Band Calculation LSU ff_23p25_eps_10p89"
postprocess_results = []
runtime_ps = 9e-12
min_steps_per_lambda = 20
cuts = [1]
h5_bg = None
ref = True
run = False
if os.path.isfile(filename):
    structure_1 = AM.loadAndRunStructure(
                key = tidy3dAPI, file_path=filename
                ,direction="z", lambda_range=lambdas,
                box_size=14.3,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
                scaling=1,shuoff_condtion=1e-20, verbose=True, 
                monitors=["flux"], flux_monitor_position=18,cell_size_manual=45,width=0.5,
                freqs=300, 
                cut_condition=1, source="planewave", absorbers=130, use_permittivity=False,sim_name=rf"{Path(filename).stem}_size_{1}" + (rf"_bg_{h5_bg}" if h5_bg else ""),h5_bg=h5_bg,
                )

               


In [ ]:
sim = structure_1.sim
sim = sim.copy(update={'sources':[]})
sim = sim.copy(update={'monitors':[]})

In [ ]:
num_dipoles = 10
num_monitors = 3
dipole_positions = rng.uniform(
    [-structure_1.Lx / 2, -structure_1.Ly / 2, 0], [structure_1.Lx / 2, structure_1.Ly / 2, 0], [num_dipoles, 3]
)

dipole_phases = rng.uniform(0, 2 * np.pi, num_dipoles)

pulses = []
dipoles = []
for i in range(num_dipoles):
    pulse = td.GaussianPulse(freq0=structure_1.freq0, fwidth=structure_1.freqw, phase=dipole_phases[i])
    pulses.append(pulse)
    dipoles.append(
        td.PointDipole(
            source_time=pulse,
            center=tuple(dipole_positions[i]),
            polarization="Hz",
            name="dipole_" + str(i),
        )
    )

In [ ]:
monitor_positions = rng.uniform(
    [-structure_1.Lx / 2, -structure_1.Ly / 2, 0], [structure_1.Lx / 2, structure_1.Ly / 2, 0], [num_monitors, 3]
)
t_start_fwidth = 5.0
t_start = t_start_fwidth / structure_1.freqw
monitors_time = []
for i in range(num_monitors):
    monitors_time.append(
        td.FieldTimeMonitor(
            fields=["Hz"],
            center=tuple(monitor_positions[i]),
            size=(0, 0, 0),
            start=t_start,
            name="monitor_time_" + str(i),
        )
    )

In [ ]:
# Number of k values to sample, per edge of the irreducible Brillouin zone
Nk = 4

bspecs_gammax = []
bspecs_xm = []
bspecs_mr = []
bspecs_rgamma = []
for i in range(Nk):
    bspecs_gammax.append(
        td.BoundarySpec(
            x=td.Boundary.bloch((1 / 2) * i / Nk),
            y=td.Boundary.periodic(),
            z=td.Boundary.pml(),
        )
    )
    bspecs_xm.append(
        td.BoundarySpec(
            x=td.Boundary.bloch(1 / 2),
            y=td.Boundary.bloch((1 / 2) * i / Nk),
            z=td.Boundary.pml(),
        )
    )
    bspecs_mr.append(
        td.BoundarySpec(
            x=td.Boundary.bloch(1 / 2),
            y=td.Boundary.bloch(1 / 2),
            z=td.Boundary.bloch((1 / 2) * i / Nk),
        )
    )
    bspecs_rgamma.append(
        td.BoundarySpec(
            x=td.Boundary.bloch((1 / 2) * (1 - i / Nk)),
            y=td.Boundary.bloch((1 / 2) * (1 - i / Nk)),
            z=td.Boundary.bloch((1 / 2) * (1 - i / Nk)),
        )
    )
bspecs = bspecs_gammax + bspecs_xm + bspecs_mr + bspecs_rgamma

In [ ]:
sims = {}
for i in range(4 * Nk):
    sims[f"sim_{i}"] = sim.copy(update={'sources':dipoles,"monitors":monitors_time,"boundary_spec":bspecs[i],"symmetry": (0, 0, 0),"normalize_index":None,"shutoff":0})

In [ ]:
fig, ax = plt.subplots(1, 2, tight_layout=True, figsize=(10, 4))
sims["sim_0"].plot(z=0.0, ax=ax[0])
sims["sim_0"].plot(x=0, freq=structure_1.freq0, ax=ax[1])
plt.show()
sims["sim_0"].plot_eps(x=0, freq=structure_1.freq0)

In [ ]:
batch_data={}
for item in sims.keys():
    file_desc = rf"H:\phd stuff\tidy3d\data\{project_name}\{item}.txt"
    if os.path.exists(file_desc):
        print("Exist!")
        item_sim = AM.loadFromFile(key = tidy3dAPI, file_path=file_desc).sim_data
    else:
        print("Creating...")
        id =web.upload(sims[item], folder_name=project_name,task_name=item, verbose=True)
        web.start(task_id = id)
        ids = '\n' + id
        # Check if the folder exists
        os.makedirs(rf"H:\phd stuff\tidy3d\data\{project_name}",exist_ok=True)
        # Open file in write mode
        with open(file_desc, "w") as f:
            # Write the string to the file
            f.write(ids)
        web.monitor(id)

        item_sim = AM.loadFromFile(key = tidy3dAPI, file_path=file_desc).sim_data
    
    
    batch_data[item] = item_sim

In [ ]:
plt.plot(
    batch_data["sim_1"]["monitor_time_0"].Hz.t,
    np.real(batch_data["sim_1"].monitor_data["monitor_time_0"].Hz.squeeze()),
)
plt.title("FieldTimeMonitor data")
plt.xlabel("t")
plt.ylabel("Hz")
plt.show()


In [ ]:
freq_range = td.C_0/lambdas
field = batch_data["sim_0"].monitor_data["monitor_time_0"].Hz.squeeze().real

fmesh = np.fft.fftfreq(field.size, np.mean(np.diff(field.t)))
spectrum = np.fft.fft(field)

mask = (fmesh > freq_range[0]) & (fmesh < freq_range[1])

plt.plot(
    fmesh[mask],
    np.abs(spectrum[mask]),
)
plt.title("Spectrum at single wavevector")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.yscale("log")
plt.show()


In [ ]:
resonance_finder = ResonanceFinder(freq_window=tuple(freq_range))
resonance_datas = []
for i in range(4 * Nk):
    sim_data = batch_data[f"sim_{i}"]
    resonance_datas.append(resonance_finder.run(signals=sim_data.data))

In [ ]:
def filter_resonances(resonance_data, minQ, minamp, maxerr):
    resonance_data = resonance_data.where(abs(resonance_data.Q) > minQ, drop=True)
    resonance_data = resonance_data.where(resonance_data.amplitude > minamp, drop=True)
    resonance_data = resonance_data.where(resonance_data.error < maxerr, drop=True)
    return resonance_data

In [ ]:
all_lambdas = []
all_vectors = []
a=2.04
for i in range(4 * Nk):
    resonance_data = resonance_datas[i]
    resonance_data = filter_resonances(
        resonance_data=resonance_data, minQ=0, minamp=1e-5, maxerr=1
    )
    freqs_i = resonance_data.freq.to_numpy()
    lambdas_i = 0.8*td.C_0/freqs_i
    Qs = resonance_data.Q.to_numpy()
    all_lambdas.append(lambdas_i)
    all_vectors.append((1 / 2) * i / Nk)
    plt.scatter(np.full(len(freqs_i), (1 / 2) * i / Nk),a/lambdas_i, color="blue")


# Nk: number of k-points per segment
k_tick_positions = [(1/2) * i / Nk for i in [0, Nk, 2*Nk, 3*Nk,4*Nk-1]]
k_tick_labels = [r'$\Gamma$', r'$X$', r'$M$',r'$R$',r'$\Gamma$']

#Find the gap 

flattened_lambdas = np.sort(np.concatenate(all_lambdas))
gaps = np.diff(flattened_lambdas)
max_gap_index = np.argmax(gaps)
for i, item in enumerate(gaps):
    if item>0.3:
        bandgap_lower = flattened_lambdas[i]
        bandgap_upper = flattened_lambdas[i + 1]
        gap_width = (bandgap_upper-bandgap_lower)/((bandgap_upper+bandgap_lower)/2)
        plt.fill_between(all_vectors, a/bandgap_lower, a/bandgap_upper, color='red', alpha=0.2)
        plt.text(a/bandgap_lower, a/bandgap_upper, rf'$\Delta \omega / \omega_0 = {gap_width*100 :.2f}$%', fontsize = 16)


#################
plt.xticks(k_tick_positions, k_tick_labels)
plt.ylabel(r"$a/\lambda$")
plt.title(rf"eps {structure_1.permittivity_value:.2f} L=11.44, ff_23p25")
# plt.ylim(0.35,0.5)
plt.show()